In [85]:
import pandas as pd
import duckdb as dd
import sys
import os

carpeta = "TablasOriginales"

padron_poblacion =  pd.read_excel(carpeta + "/padron_poblacion.xlsX")
datos_departamento = pd.read_csv(carpeta + "/Datos_por_departamento_actividad_y_sexo.csv")
padron_establecimientos_educativos  = pd.read_excel(carpeta + "/2022_padron_oficial_establecimientos_educativos.xlsx", skiprows = 6)
actividades_establecimientos_productivos = pd.read_csv(carpeta + "/actividades_establecimientos.csv")



# Limpieza padron_poblacion


In [86]:
carpeta = "TablasOriginales"

padron_poblacion =  pd.read_excel(carpeta + "/padron_poblacion.xlsX")

padron_poblacion.columns = ['nada', 'edad', 'casos', 'porcentaje', 'porcentaje_acumulado']
padron_poblacion.columns


edad_index = []
total_index = []
areas = []
id_areas = []
counter = 0
counter_total = 0


for i in range(len(padron_poblacion['edad'].tolist())): # saco los indices de fila donde aparece Edad 
    if padron_poblacion['edad'].tolist()[i] == 'Edad':
        counter += 1
        edad_index.append(i+1)
        id_areas.append(padron_poblacion['edad'].tolist()[i-2])
        areas.append(padron_poblacion['casos'].tolist()[i-2])


for i in range(len(padron_poblacion['edad'].tolist())): # saco los indices de fila donde aparece Total
    if padron_poblacion['edad'].tolist()[i] == 'Total':
        total_index.append(i)
        counter_total += 1

if  counter != counter_total:
    print("Error: Tamaños de indexacion diferentes")
    sys.exit()


df_list = []
for index in range(counter): #
    edad = padron_poblacion['edad'][edad_index[index]: total_index[index]].tolist()
    casos = padron_poblacion['casos'][edad_index[index]: total_index[index]].tolist()
    porcentaje = padron_poblacion['porcentaje'][edad_index[index]: total_index[index]].tolist()
    porcentaje_acumulado = padron_poblacion['porcentaje_acumulado'][
        edad_index[index]: total_index[index]].tolist()
    
    data = {
        'edad': edad,
        'casos': casos,
        'porcentaje': porcentaje,
        'porcentaje_acumulado': porcentaje_acumulado
    }
    df_ = pd.DataFrame(data)

    ruta_subcarpeta = os.path.join('TablasModificadas', 'Padrones')

    os.makedirs(ruta_subcarpeta, exist_ok=True)


    id_departamentos = [id_area.replace('AREA # ', '') for id_area in id_areas]
    ruta_archivo = os.path.join(ruta_subcarpeta, f'padron_poblacion_{id_departamentos[index]}.csv')
    df_.to_csv(ruta_archivo, index=False)
    
    df_list.append(df_)




In [87]:
areas = areas[:-1]
id_areas = id_areas[:-1] 

print( id_areas[-1])

AREA # 94015


In [88]:
id_departamentos = [id_area.replace('AREA # ', '') for id_area in id_areas]

# Crear el mapeo correcto
mapeo_departamentos = pd.DataFrame({
    'id_departamentos': id_departamentos,
    'departamento': areas
})

mapeo_departamentos


,id_departamentos,departamento
0,02007,Comuna 1
1,02014,Comuna 2
2,02021,Comuna 3
3,02028,Comuna 4
4,02035,Comuna 5
...,...,...
522,90112,Trancas
523,90119,Yerba Buena
524,94008,Río Grande
525,94011,Tolhuin


In [89]:
# remuevo el resumen:
ruta_archivo = os.path.join('TablasModificadas', 'Padrones', 'padron_poblacion_RESUMEN.csv')
os.remove(ruta_archivo)

In [91]:
ruta_padrones = os.path.join('TablasModificadas', 'Padrones')
df_grupos_etarios_ = []

archivos_padrones = [f for f in os.listdir(ruta_padrones) if f.startswith('padron_poblacion_') and f.endswith('.csv')]


for archivo in archivos_padrones:
    id_departamento = archivo.replace('padron_poblacion_', '').replace('.csv', '')
    ruta_completa = os.path.join(ruta_padrones, archivo)
    df_departamento = pd.read_csv(ruta_completa)
    id_departamento = id_departamento.replace("'", "''")
    consultaSQL = f"""
    SELECT 
        '{id_departamento}' AS id_departamento,
        SUM(CASE WHEN edad BETWEEN 0 AND 2 THEN casos ELSE 0 END) AS cant_edad_prescolar,
        SUM(CASE WHEN edad BETWEEN 3 AND 5 THEN casos ELSE 0 END) AS cant_edad_jardin,
        SUM(CASE WHEN edad BETWEEN 6 AND 11 THEN casos ELSE 0 END) AS cant_edad_primaria,
        SUM(CASE WHEN edad BETWEEN 12 AND 17 THEN casos ELSE 0 END) AS cant_edad_secundaria,
        SUM(CASE WHEN edad BETWEEN 18 AND 64 THEN casos ELSE 0 END) AS cant_edad_laboral,
        SUM(CASE WHEN edad >= 65 THEN casos ELSE 0 END) AS cant_edad_jubilatoria
    FROM df_departamento
    ORDER BY id_departamento ASC
    """
    
    dataframeResultado = dd.sql(consultaSQL).df()
    df_grupos_etarios_.append(dataframeResultado)

df_grupos_etarios = pd.concat(df_grupos_etarios_, ignore_index=True)

df_grupos_etarios


,id_departamento,cant_edad_prescolar,cant_edad_jardin,cant_edad_primaria,cant_edad_secundaria,cant_edad_laboral,cant_edad_jubilatoria
0,06693,489.0,615.0,1342.0,1352.0,8234.0,1869.0
1,06371,12141.0,15897.0,38161.0,39283.0,275789.0,63232.0
2,06147,752.0,979.0,2127.0,2160.0,13633.0,3254.0
3,22112,991.0,1099.0,2597.0,2592.0,12689.0,1576.0
4,06525,2513.0,3460.0,7274.0,7274.0,38901.0,5339.0
...,...,...,...,...,...,...,...
522,90077,2514.0,3011.0,6593.0,6541.0,39267.0,7001.0
523,66049,2159.0,2699.0,5990.0,5543.0,33982.0,5373.0
524,82098,1454.0,1622.0,3454.0,3007.0,20208.0,3435.0
525,58014,884.0,1024.0,2170.0,1931.0,11265.0,876.0


In [92]:
consultaSQL = """ SELECT id_departamento, mapeo_departamentos.departamento AS departamento_nombre , cant_edad_prescolar, cant_edad_jardin, cant_edad_primaria,cant_edad_secundaria, cant_edad_laboral,cant_edad_jubilatoria,

FROM df_grupos_etarios INNER JOIN mapeo_departamentos ON  df_grupos_etarios.id_departamento = mapeo_departamentos.id_departamentos



"""


df_depa_con_nombre = dd.sql(consultaSQL).df()
df_depa_con_nombre


,id_departamento,departamento_nombre,cant_edad_prescolar,cant_edad_jardin,cant_edad_primaria,cant_edad_secundaria,cant_edad_laboral,cant_edad_jubilatoria
0,06693,Roque Pérez,489.0,615.0,1342.0,1352.0,8234.0,1869.0
1,06371,General San Martín,12141.0,15897.0,38161.0,39283.0,275789.0,63232.0
2,06147,Carlos Casares,752.0,979.0,2127.0,2160.0,13633.0,3254.0
3,22112,O'Higgins,991.0,1099.0,2597.0,2592.0,12689.0,1576.0
4,06525,Marcos Paz,2513.0,3460.0,7274.0,7274.0,38901.0,5339.0
...,...,...,...,...,...,...,...,...
522,90077,Río Chico,2514.0,3011.0,6593.0,6541.0,39267.0,7001.0
523,66049,General Güemes,2159.0,2699.0,5990.0,5543.0,33982.0,5373.0
524,82098,San Javier,1454.0,1622.0,3454.0,3007.0,20208.0,3435.0
525,58014,Añelo,884.0,1024.0,2170.0,1931.0,11265.0,876.0


In [93]:
consultaSQL = """   SELECT DISTINCT provincia, id_departamento, departamento_nombre, cant_edad_prescolar, cant_edad_jardin, cant_edad_primaria,cant_edad_secundaria, cant_edad_laboral,cant_edad_jubilatoria,


FROM df_depa_con_nombre LEFT JOIN datos_departamento ON  datos_departamento.in_departamentos = df_depa_con_nombre.id_departamento

ORDER BY id_departamento ASC

"""

df_final = dd.sql(consultaSQL).df()

# Meto algunos datos a mano porque esta imposible sino las ids no arrancan

df_final.loc[45, 'provincia'] = 'Buenos Aires'  
df_final.loc[524, 'provincia'] = 'Tierra Del Fuego'
df_final.loc[526, 'provincia'] = 'Tierra Del Fuego'

df_final

,provincia,id_departamento,departamento_nombre,cant_edad_prescolar,cant_edad_jardin,cant_edad_primaria,cant_edad_secundaria,cant_edad_laboral,cant_edad_jubilatoria
0,CABA,02007,Comuna 1,5368.0,6851.0,14886.0,14159.0,148749.0,30988.0
1,CABA,02014,Comuna 2,3095.0,3521.0,7503.0,7822.0,106104.0,32564.0
2,CABA,02021,Comuna 3,4021.0,5197.0,11820.0,12007.0,129544.0,30948.0
3,CABA,02028,Comuna 4,5724.0,8119.0,19242.0,18847.0,144071.0,31021.0
4,CABA,02035,Comuna 5,3669.0,4779.0,11151.0,11315.0,126865.0,34670.0
...,...,...,...,...,...,...,...,...,...
522,Tucuman,90112,Trancas,1022.0,1163.0,2451.0,2433.0,14035.0,2334.0
523,Tucuman,90119,Yerba Buena,3731.0,4689.0,10227.0,9933.0,64306.0,9703.0
524,Tierra Del Fuego,94008,Río Grande,3316.0,4771.0,10167.0,9942.0,63994.0,6732.0
525,Tierra Del Fuego,94011,Tolhuin,259.0,322.0,715.0,660.0,3697.0,374.0


In [94]:
ruta = os.path.join('TablasModificadas')
os.makedirs(ruta, exist_ok=True)
ruta_archivo = os.path.join(ruta, 'dato_padron_por_departamento.csv')
df_final.to_csv(ruta_archivo, index=False)

# Limpieza EP

In [95]:
consultaSQL = """
SELECT
    datos_departamento.clae6,
    genero,
    in_departamentos AS id_departamentos,
    empresas_exportadoras,
    establecimientos AS establecimientos,
    empleo AS empleados,
    clae2_desc
FROM datos_departamento

INNER JOIN  actividades_establecimientos_productivos 
ON datos_departamento.clae6 = actividades_establecimientos_productivos.clae6
WHERE anio = 2022
              """

datos_establecimientos= dd.sql(consultaSQL).df()

datos_establecimientos

,clae6,genero,id_departamentos,empresas_exportadoras,establecimientos,empleados,clae2_desc
0,432920,Mujeres,82119,0,1,2,Actividades especializadas de construcción
1,432990,Varones,82119,0,8,190,Actividades especializadas de construcción
2,432990,Mujeres,82119,0,3,16,Actividades especializadas de construcción
3,433010,Varones,82119,0,1,3,Actividades especializadas de construcción
4,433040,Varones,82119,0,3,29,Actividades especializadas de construcción
...,...,...,...,...,...,...,...
160476,432190,Varones,82119,0,31,260,Actividades especializadas de construcción
160477,432190,Mujeres,82119,0,7,11,Actividades especializadas de construcción
160478,432200,Varones,82119,0,7,18,Actividades especializadas de construcción
160479,432200,Mujeres,82119,0,2,3,Actividades especializadas de construcción


In [96]:

ruta = os.path.join('TablasModificadas')
os.makedirs(ruta, exist_ok=True)
ruta_archivo = os.path.join(ruta, 'datos_establecimientos.csv')
datos_establecimientos.to_csv(ruta_archivo, index=False)


# Limpieza EE

In [99]:
hacer

NameError: name 'hacer' is not defined

In [97]:
padron_establecimientos_educativos  = pd.read_excel(carpeta + "/2022_padron_oficial_establecimientos_educativos.xlsx", skiprows = 6) # salto los irrelevantes
# saquemosle los espacios para poder llamarlos con sql despues:
padron_establecimientos_educativos.columns=['Jurisdicción', 'Cueanexo', 'Nombre', 'Sector', 'Ámbito', 'Domicilio',
       'C_P', 'Código_de_área', 'Teléfono', 'Código_de_localidad',
       'Localidad', 'departamento', 'Mail', 'Común', 'Especial', 'Adultos',
       'Artística', 'Hospitalaria', 'Intercultural', 'Encierro',
       'Jardín_maternal', 'Jardín_de_infantes',
       'Primario', 'Secundario', 'Secundario_INET', 'SNU', 'SNU_INET',
       'Secundario.1', 'SNU.1', 'Talleres',
       'Nivel_inicial_Educación_temprana',
       'Nivel_inicial_Jardín_de_infantes.1', 'Primario.1', 'Secundario.2',
       'Integración_a_la_modalidad_común/_adultos', 'Primario.2',
       'Secundario.3', 'Alfabetización', 'Formación_Profesional',
       'Formación_Profesional_INET', 'Inicial', 'Primario.3', 'Secundario.4',
       'Unnamed: 43']
print(padron_establecimientos_educativos.columns)



Index(['Jurisdicción', 'Cueanexo', 'Nombre', 'Sector', 'Ámbito', 'Domicilio',
       'C_P', 'Código_de_área', 'Teléfono', 'Código_de_localidad', 'Localidad',
       'departamento', 'Mail', 'Común', 'Especial', 'Adultos', 'Artística',
       'Hospitalaria', 'Intercultural', 'Encierro', 'Jardín_maternal',
       'Jardín_de_infantes', 'Primario', 'Secundario', 'Secundario_INET',
       'SNU', 'SNU_INET', 'Secundario.1', 'SNU.1', 'Talleres',
       'Nivel_inicial_Educación_temprana',
       'Nivel_inicial_Jardín_de_infantes.1', 'Primario.1', 'Secundario.2',
       'Integración_a_la_modalidad_común/_adultos', 'Primario.2',
       'Secundario.3', 'Alfabetización', 'Formación_Profesional',
       'Formación_Profesional_INET', 'Inicial', 'Primario.3', 'Secundario.4',
       'Unnamed: 43'],
      dtype='object')


In [98]:
len(set(mapeo_departamentos['id_departamentos'].to_list()))
# TAbla relaciones cuanexo_nivel
#%%--------------------------------------------------------------------------------------------------------------------------------
tipos_educativos = [
    'Jardín_maternal',
    'Jardín_de_infantes', 
    'Primario',
    'Secundario',
    'Secundario_INET',
    'SNU',
    'SNU_INET'
]

partes_sql = []

for tipo in tipos_educativos:
    parte = f"""
    SELECT Cueanexo, '{tipo}' as tipo_establecimiento
    FROM padron_establecimientos_educativos 
    WHERE (Común = '1' OR Formación_Profesional = '1' OR Formación_Profesional_INET = '1')
      AND {tipo} = '1'
    """
    partes_sql.append(parte)

consulta_union_completa = " UNION ALL ".join(partes_sql)

tabla_de_relaciones = dd.sql(consulta_union_completa).df()
tabla_de_relaciones


,Cueanexo,tipo_establecimiento
0,20000100,Jardín_maternal
1,20000300,Jardín_maternal
2,20000500,Jardín_maternal
3,20000700,Jardín_maternal
4,20001700,Jardín_maternal
...,...,...
65828,900210300,SNU_INET
65829,940003200,SNU_INET
65830,940006500,SNU_INET
65831,940006504,SNU_INET
